In [6]:
# This python program connect to my blockchain database and generate the schema for the indexer project
#This should create a total of 37 tables
#Includes three tables types, block table, transaction header table, transaction detail table

import psycopg2
import logging

DB_NAME = 'blockchain'
DB_USER = 'postgres'
DB_PASSWORD = 'auth2020'
DB_HOST = 'localhost'
DB_PORT = '5432'


try:
    # Create a PostgreSQL connection
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print("Connection established.")  # Log successful connection (you can modify this)

    # Configure logging to write to a log file
    logging.basicConfig(filename='log.txt', level=logging.INFO)
    logging.info("Connection established with Database.")  # Log the same message to the file


    # Additional code for database operations goes here...

except Exception as err:
    # Handle exceptions
    print("Unable to connect to database.")  # Log error (you can modify this)
    print(err)  # Print the specific error message

    # Optionally, write the error message to a file (error.txt)
    with open('error.txt', 'a') as error_file:
        error_file.write(str(err) + '\n')
        
        

Connection established.


In [7]:
#SCHEMA CREATION
import psycopg2
def create_tables():
    commands = (
        # Table for primary ledger
        """
        CREATE TABLE block (
            block_id  UUID DEFAULT gen_random_uuid() PRIMARY KEY NOT NULL,
            block VARCHAR NOT NULL,
            type VARCHAR NOT NULL
        )
        """,
        # Transaction header table for type "exec"
        """
        CREATE TABLE transaction_header_exec (
            transaction_exec_id  UUID DEFAULT gen_random_uuid() PRIMARY KEY NOT NULL ,
            block_id  UUID NOT NULL,
            type VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        """
        CREATE TABLE transaction_detail_msg_exec (
            msg_exec_id  UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_exec_id UUID NOT NULL,
            type VARCHAR NOT NULL,
            delegator_address VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            FOREIGN KEY(transaction_exec_id) REFERENCES transaction_header_exec(transaction_exec_id)
        )
        """,
        """
        CREATE TABLE transaction_header_update (
            transaction_update_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,    
        """
        CREATE TABLE transaction_detail_msgupdate (
            msg_update_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_update_id UUID NOT NULL,
            client_id VARCHAR NOT NULL,
            signer VARCHAR NOT NULL,
            FOREIGN KEY(transaction_update_id) REFERENCES transaction_header_update(transaction_update_id)
            
        )
        """,
        """
        CREATE TABLE transaction_detail_msg_packet (
            msg_packet_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_update_id UUID NOT NULL,
            Signer VARCHAR NOT NULL,
            packet_sequence INTEGER NOT NULL,
            packet_source_port VARCHAR NOT NULL,
            transfer VARCHAR NOT NULL,
            packet_source_channel VARCHAR NOT NULL,
            packet_destination_port VARCHAR NOT NULL,
            packet_destination_channel VARCHAR NOT NULL,
            packet_Data VARCHAR NOT NULL,
            proof_commitment VARCHAR NOT NULL,
            proof_revision_number INTEGER NOT NULL,
            proof_height INTEGER NOT NULL,
            FOREIGN KEY(transaction_update_id) REFERENCES transaction_header_update(transaction_update_id)
        )
        """,
        """
        CREATE TABLE transaction_detail_msg_acknowledgement (
            msg_acknowledgement_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_update_id UUID NOT NULL,
            packet_sequence VARCHAR NOT NULL,
            packet_source_channel VARCHAR NOT NULL,
            packet_destination_port VARCHAR NOT NULL,
            packet_destination_channel VARCHAR NOT NULL,
            packet_data VARCHAR NOT NULL,
            proof_number_height INTEGER NOT NULL,
            proof_height VARCHAR NOT NULL,
            FOREIGN KEY(transaction_update_id) REFERENCES transaction_header_update(transaction_update_id)
            )
        """,
        """
        CREATE TABLE transaction_header_transfer (
            transaction_transfer_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        """
        CREATE TABLE transaction_detail_msg_transfers (
            msg_transfer_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_transfer_id UUID NOT NULL,
            sender VARCHAR NOT NULL,
            receiver VARCHAR NOT NULL,
            source_port VARCHAR NOT NULL,
            source_channel VARCHAR NOT NULL,
            FOREIGN KEY(transaction_transfer_id) REFERENCES transaction_header_transfer(transaction_transfer_id)
        )
        """,
        """
        CREATE TABLE transaction_header_execute (
            transaction_execute_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #transaction detail table for msg "ExecuteContract"
        """
        CREATE TABLE transaction_detail_msg_executecontract (
            msg_execute_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_execute_id UUID NOT NULL,
            sender VARCHAR NOT NULL,
            contract VARCHAR NOT NULL,
            FOREIGN KEY(transaction_execute_id) REFERENCES transaction_header_execute(transaction_execute_id)
            
            
        )
        """,
        # transaction header table for type "Delegate"
        """
        CREATE TABLE transaction_header_delegate (
            transaction_delegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            type VARCHAR,
            denom VARCHAR,
            amount INT,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        #transaction detail table for msg "cosmos_MsgDelegate"
         """
        CREATE TABLE transaction_detail_c_msg_delegate(
            msg_c_delegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_delegate_id UUID NOT NULL,
            amount INT NOT NULL,
            denom VARCHAR NOT NULL,
            delegator_address VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL,
            FOREIGN KEY(transaction_delegate_id) REFERENCES transaction_header_delegate(transaction_delegate_id)          
            
        )
        """,
        # transaction detail table for "alliance_MsgDelegate"
        """
        CREATE TABLE transaction_detail_a_msg_delegate(
            msg_a_delegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_delegate_id UUID NOT NULL,
            amount INT NOT NULL,
            denom VARCHAR NOT NULL,
            delegator_address VARCHAR NOT NULL,
            validator_address VARCHAR NOT NULL, 
            FOREIGN KEY(transaction_delegate_id) REFERENCES transaction_header_delegate(transaction_delegate_id)                
            
        )
        """,
        #Transaction header table type "withdraw"
        """
        CREATE TABLE transaction_header_withdraw (
            transaction_withdraw_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        #Transaction detail table msg "WithdrawDelegatorReward"
        """
        CREATE TABLE transaction_detail_msg_withdraw_delegator_reward(
            msg_delegator_reward_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_withdraw_id UUID NOT NULL,
            delegator_Address VARCHAR NOT NULL,
            validator_Address VARCHAR NOT NULL,
            reward VARCHAR NOT NULL,
            FOREIGN KEY(transaction_withdraw_id) REFERENCES transaction_header_withdraw(transaction_withdraw_id)
            
        )
        """,
        """
        CREATE TABLE transaction_header_validator_commission (
            transaction_commission_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        """
        CREATE TABLE transaction_detail_msg_withdraw_validator_commission(
            msg_validator_commission_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_commission_id UUID NOT NULL,
            validator_address VARCHAR NOT NULL,
            FOREIGN KEY(transaction_commission_id) REFERENCES transaction_header_Validator_Commission(transaction_commission_id)
            
        )
        """,
        # transaction header table for type "UnDelegate"
        """
        CREATE TABLE transaction_header_undelegate (
            transaction_undelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)

        )
        """,
        #Transaction detail table for msg "cosmos_MsgUndelegate"
        """
        CREATE TABLE transaction_detail_c_msg_undelegate(
            msg_c_undelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_undelegate_id UUID NOT NULL,
            delegator_Address VARCHAR NOT NULL,
            validator_Address VARCHAR NOT NULL,
            amount VARCHAR NOT NULL,
            FOREIGN KEY(transaction_undelegate_id) REFERENCES transaction_header_UnDelegate(transaction_undelegate_id)
            
            
        )
        """,
        #Transaction detail table for msg "alliance_MsgUndelegate"
        """
        CREATE TABLE transaction_detail_a_msg_undelegate(
            msg_a_undelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_undelegate_id UUID NOT NULL,
            delegator_Address VARCHAR NOT NULL,
            validator_Address VARCHAR NOT NULL,
            amount VARCHAR NOT NULL,
            FOREIGN KEY(transaction_undelegate_id) REFERENCES transaction_header_UnDelegate(transaction_undelegate_id)
        )
        """,
        #Transaction header table type "Send"
        """
        CREATE TABLE transaction_header_send (
            transaction_send_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE transaction_detail_msg_send(
            msg_send_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_send_id UUID NOT NULL,
            sender VARCHAR NOT NULL,
            receiver VARCHAR NOT NULL,
            raw_data JSONB NOT NULL,
            FOREIGN KEY(transaction_send_id) REFERENCES transaction_header_Send(transaction_send_id)
            
            
        )
        """,
        #Transaction header table type "ClaimDelegate"
        """
        CREATE TABLE transaction_header_claim_delegate (
            transaction_claim_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE transaction_detail_msg_claim_delegation_rewards(
            msg_claim_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_claim_id UUID NOT NULL,
            FOREIGN KEY(transaction_claim_id) REFERENCES transaction_header_claim_delegate(transaction_claim_id)
        )
        """,
        #Transaction header table type "Grant"
        """
        CREATE TABLE transaction_header_grant(
            transaction_grant_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE transaction_detail_msg_grant(
            msg_grant_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_grant_id UUID NOT NULL,
            granter VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            FOREIGN KEY(transaction_grant_id) REFERENCES transaction_header_Grant(transaction_grant_id)
            
        )
        """,
        #Transaction header table type "BeginDelegate"
        """
        CREATE TABLE transaction_header_begin_redelegate(
            transaction_begin_redelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE transaction_detail_msg_begin_redelegate(
            msg_begin_redelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_begin_redelegate_id UUID NOT NULL,
            delegator_address VARCHAR NOT NULL,
            validator_source_address VARCHAR NOT NULL,
            validator_target_address VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            denom VARCHAR NOT NULL,
            FOREIGN KEY(transaction_begin_redelegate_id) REFERENCES transaction_header_begin_redelegate(transaction_begin_redelegate_id)
            
        )
        """,
        #Transaction header table type "Revoke"
        """
        CREATE TABLE transaction_header_revoke(
            transaction_revoke_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE transaction_detail_msg_revoke(
            msg_revoke_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_revoke_id UUID NOT NULL,
            granter VARCHAR NOT NULL,
            grantee VARCHAR NOT NULL,
            FOREIGN KEY(transaction_revoke_id) REFERENCES transaction_header_revoke(transaction_revoke_id)
            
        )
        """,
        #Transaction header table type "StoreCode"
        """
        CREATE TABLE transaction_header_storecode(
            transaction_storecode_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE transaction_detail_msg_storecode(
            msg_store_code_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_storecode_id UUID NOT NULL,
            FOREIGN KEY(transaction_storecode_id) REFERENCES transaction_header_storecode(transaction_storecode_id)
            
        )
        """,
        #Transaction header table type "InstantiateContract"
        """
        CREATE TABLE transaction_header_instantiate_contract(
            transaction_instantiate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
        )
        """,
        #Find Transaction block
        """
        CREATE TABLE transaction_detail_msg_instantiate_contract(
            msg_instantiate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_instantiate_id UUID NOT NULL,
            FOREIGN KEY(transaction_instantiate_id) REFERENCES transaction_header_instantiate_contract(transaction_instantiate_id)
        )
        """,
        #Transaction header table for type "Redelegate"
        """
        CREATE TABLE transaction_header_redelegate(
            transaction_redelegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,

        """
        CREATE TABLE transaction_detail_msg_redelegate(
            msg_redegegate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_redelegate_id UUID NOT NULL,
            FOREIGN KEY(transaction_redelegate_id) REFERENCES transaction_header_redelegate(transaction_redelegate_id)
            
        )
        """,
        
        """
        CREATE TABLE transaction_header_vote(
            transaction_vote_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            block_id  UUID NOT NULL,
            denom VARCHAR NOT NULL,
            amount INTEGER NOT NULL,
            gas_limit INTEGER NOT NULL,
            FOREIGN KEY(block_id) REFERENCES block(block_id)
            
        )
        """,
        """
        CREATE TABLE transaction_detail_msg_vote(
            msg_vote_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
            transaction_vote_id UUID NOT NULL,
            proposal_id INT NOT NULL,
            voter VARCHAR NOT NULL,
            option VARCHAR NOT NULL,
            FOREIGN KEY(transaction_vote_id) REFERENCES transaction_header_vote(transaction_vote_id)
            
        )
        """,
        """
        CREATE TABLE transaction_header_migrate_contract (
        transaction_migrate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
        block_id UUID NOT NULL,
        denom VARCHAR NOT NULL,
        amount INTEGER NOT NULL,
        gas_limit INTEGER NOT NULL,
        FOREIGN KEY(block_id) REFERENCES block(block_id)
        
        )
        """,
        """
        CREATE TABLE transaction_detail_msg_migrate(
        msg_migrate_id UUID DEFAULT gen_random_uuid() NOT NULL PRIMARY KEY,
        transaction_migrate_id UUID NOT NULL,
        sender VARCHAR NOT NULL,
        contract VARCHAR NOT NULL,
        code_id VARCHAR,
        FOREIGN KEY(transaction_migrate_id) REFERENCES transaction_header_migrate_contract(transaction_migrate_id)
        )
        """
        
                # Transaction detail table for msg "exec"       
    )

    try:
        # Replace with your actual database connection parameters
        conn = psycopg2.connect(
            dbname='blockchain',
            user='postgres',
            password='auth2020',
            host='localhost',
            port='5432'
        )

        with conn:
            with conn.cursor() as cur:
                for command in commands:
                    cur.execute(command)

    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

if __name__ == '__main__':
    create_tables()     